Módulos Python
==============

![dependencias modulos en Python](../images/04%20dependencias%20modulos.png)

### Creación módulos

Escribe la siguiente función en un fichero llamado `modulo.py` para crear un objeto módulo con un único atributo llamado `printer`.

In [ ]:
def printer(x):     # Module attribute printer
    print(x)

## Uso de módulos

Un cliente puede usar el módulo `modulo` que acabamos de crear ejecutando una sentencia `import` o `from`. 

Ambas sentencias encuentran, compilan y ejecutan el código del módulo, sólo si no ha sido cargado previamente.

`import` carga el modulo como un conjunto, por lo que debes incluir el nombre del módulo cuando nombras a sus atributos; por el contrario, `from` carga (o copia) nombres específicos del módulo.

### sys.path

Cuando ejecuto un programa python el directorio desde el que se ejecuta la búsqueda de modulos es el directorio desde el que se invoca en consola `python3`.

La búsqueda de módulos se inicializa cuando se inicia Python. Se puede acceder a esta ruta de búsqueda de módulos mediante `sys.path`.

Añado `..` a esta ruta para que las importaciones `src.modulo` se resuelvan correctamente ya que hay que "subir" hasta el directorio padre del proyecto para encontrar el directorio `src` (es imposible encontrarlo en `notebooks`).

In [ ]:
import sys
sys.path.append('..')

### Sentencia import

El nombre del módulo identifica un fichero para ser cargado, y pasa a ser una variable en el _script_.

La setencia `import` lista uno o más nombres de módulos para cargar separados por comas.

In [ ]:
import src.modulo as modulo                      # Get module as a whole (one or more)
modulo.printer("Hello world!")                   # Qualify to get names
# Hello world!

Hello world!


### Sentencia from

`from` copia nombres específicos de un fichero a otro ámbito.

`from` permite listar uno o más nombres para ser copiados separados por comas.

In [ ]:
from src.modulo import printer      # Copy out a variable (one or more) 
printer('Hello world!')             # No need to qualify name
# Hello world!

Hello world!


### Sentencia from *

Cuando usamos `*` en cambio de nombres específicos obtenemos copias de todos los nombres asignados al ámbito global del módulo referenciado.

In [ ]:
from src.modulo import *       # Copy out _all_ variables 
printer('Hello world!')
# Hello world!

Hello world!


### Sólo se importa una vez

Dado que importar es una operación cara, Python por defecto la ejecuta una única vez por fichero, por proceso. 

Importaciones sucesivas simplemente buscan el objeto que hace referencia al módulo ya cargado.

```python
# modulo_simple
print("modulo simple")
spam = 1
```

In [ ]:
import src.modulo_simple as simple   # First import: loads and runs file's code hello

modulo simple


In [ ]:
simple.spam              # the variable spam is initialized at import time
# 1

1

In [ ]:
simple.spam = 2                                 # Change attribute in module
import src.modulo_simple as simple              # Just fetches already loaded module
simple.spam                                     # Code wasn't rerun: attribute unchanged
# 2

2

In [ ]:
# Fuerzo a iPython a que resetee las variables globales y recargue los modulos. 
# Es necesario para que al "ejecutar todo" en el notebook este ejercicio y los siguientes.
# Sino, click en "Reiniciar"
# importlib.reload() recarga un modulo previamente importado.
# El argumento debe ser un objeto modulo, así que debe haber sido importado 
# previamente con exito.
import importlib
importlib.reload(simple)

modulo simple


<module 'src.modulo_simple' from '/Users/usu/Desktop/codigo/gist/python-fundamentals-nb/notebooks/../src/modulo_simple.py'>

### import y from son asignaciones

Como `def`, `import` y `from` son sentencias ejecutables, no declaraciones en tiempo de compilación, pueden:

- Anidarse en sentencias `if`; 
- Aparecer en las definiciones de funciones, para ser cargados sólo en las llamadas de la función;
- Ser usadas en sentencias `try` para proveer comportamiento por defecto.

No se resuelven o ejecutan hasta que Python las alcanza mientras se ejecuta el programa.

### Cambiando mutables en módulos

También como `def`, the `import` and `from` son asignaciones implícitas:

- `import` asigna un objeto módulo entero a un único nombre.
- `from` asigna uno o más nombres a objetos con los mismos nombres en otro módulo.

In [ ]:
# modulo_small
x = 1
y = [1, 2]

In [ ]:
from src.modulo_small import x, y       # Copy two names out
x = 42                                  # Changes local x only
y[0] = 42                               # Changes shared mutable in place
x, y

(42, [42, 2])

`x` no es un objeto compartido mutable, pero `y` sí:

In [ ]:
import src.modulo_small as small    # Get module name (from doesn't)
small.x                             # Small's x is not my x
# 1
small.x, small.y                    # But we share a changed mutable
# small.y == [42, 2]

(1, [42, 2])

#### Cambios de nombres entre ficheros

In [ ]:
from src.modulo_small import x, y       # Copy two names out 
x = 42                                  # Changes my x only
import src.modulo_small as small        # Get module name
x, small.x     # 1
# small.x = 42                          # Changes x in other module

(42, 1)

In [ ]:
importlib.reload(small)

<module 'src.modulo_small' from '/Users/usu/Desktop/codigo/gist/python-fundamentals-nb/notebooks/../src/modulo_small.py'>

### Cuándo import es requerido

La única ocasión en la que debes usar ``import`` en vez de ``from`` es cuando tienes que usar el mismo nombre definido en dos módulos distintos.

```py
# M.py
def func():
...do something...

# N.py
def func():
...do something else...
```

```py
# O.py
from M import func  
from N import func  # This overwrites the one we fetched from M func()
N.func()            # Calls N.func only!
```

```py
# O.py
import M, N         # Get the whole modules, not their names
M.func()            # We can call both names now
N.func()            # The module names make them unique           
```

### Diccionarios de espacios de nombres: __dict__

Los espacios de nombres de un módulo se almacenan como objetos diccionario.

In [ ]:
list(small.__dict__.keys())

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'x',
 'y']

Podemos filtrar sólo aquellos nombres definidos por el usuario/a, excluyendo los definidos por el _Built-in_ de Python que comienzan y terminan por los caracteres `__`

In [ ]:
list(name for name in small.__dict__.keys() if not name.startswith('__'))

['x', 'y']

## Mixed Usage Modes: __name__ and __main__

Cada módulo tienes un atributo _built-in_ llamado __name__ que Python crea y asigna automáticamente como sigue:

- Si el fichero se ejecuta como un fichero / programa _top-level_ , __name__ toma el valor string "__main__" cuando arranca.
- En cambio, si el fichero está siendo importando, __name__ toma el valor del nombre del módulo como lo conocen sus clientes.

El resultado es que un módulo puede testear su propio __name__ para determinar si está siendo ejecutado o importado.

```py
def tester():
    print("It's Christmas in Heaven...")    
if __name__ == '__main__':                  # Only when run
    tester()                                # Not when imported
```

Este módulo define una funcion para clientes que la importan y usan como de costumbre:

```sh
>>> import runme
>>> runme.tester()
It's Christmas in Heaven...
```

Pero el módulo también incluye código al pié del fichero que se configura para invocar a la función de manera automática cuando este fichero se ejecuta como un programa autónomo:

```sh
c:\code> python runme.py
It's Christmas in Heaven...
```

La variable __name__ de un módulo sirve como _flag_ de su modo de uso, permitiendo a su código ser aprovechado tanto como librería importada como _script_ de alto nivel.

__name__ funciona como código de auto-test. Puedes usar el fichero en clientes importándolo y testear su lógica ejecutándolo desde la _shell_ del sistema.